#### Завдання
### Напишіть код для визначення границь та кутів у зображенні за допомогою фільтра **Собеля**.
### Використовуючи метод густини **градієнта HOG**, напишіть код для витягування ознак із зображення.
### Завантажте датасет **CIFAR-10** та використайте **HOG** для витягування ознак зображень. Застосуйте будь-який класифікатор (наприклад **SVM**) для класифікації зображень та обчисліть такі метрики точності:
 * accuracy;
 * precision;
 * recall;
 * F1-score.
### Використайте **метод SIFT**  для витягування ознак із зображень та порівняйте його з **методом HOG** за допомогою метрик точності.